In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import clipboard
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import numpy as np

########################################
######### CHANGE THE INFO HERE #########
########################################

lg = 'A-League Women 2021-2022'
lg_id = 'd6tuztyp7kqkmikytugafu3v8' # This is the league ID from Opta.. Can find it in the html of a league's page
lg_schedule_location = "/Users/user/xG"
#lg_schedule_location = "/Users/marclamberts/Downloads"/{lg}" ## This is where you want the schedule of matches to download to
#chromium_location = "C:/Users/Ben/Coding/chromedriver.exe"  ## This is the file path to where your Chromium program is
match_download_location = "/Users/user/xG/"  ## This is the base folder that will house your league folders. Create a folder named the same as the lg variable in this filepath folder
#from pathlib import Path
#downloads_path = str(Path.home() / "Downloads")
##################################################################################
######### TO SAVE TIME, I MAKE SURE TO PUT A FILTERING MECHANISM #################
######### IN THE CODE SO THAT IT ONLY GETS GAMES AFTER THE LAST RUN DATE #########
######### ONCE YOU RUN A LEAGUE, ADD IT AND THE LAST RUN DATE ####################
######### (two examples shown commented) #########################################
##################################################################################
from datetime import datetime, timedelta
d = datetime.today() - timedelta(days=2)



if lg == 'FBL':
     last_run = "2022-09-01"
if lg == 'Indian Super League':
     last_run = "2023-02-16"
else:
    last_run = d.strftime('%Y-%m-%d')
    last_run = '2019-07-19'

##########################################################################

url = "https://api.performfeeds.com/soccerdata/match/qxcx5jmswgto1qeqcjzghtddt/?_rt=c&tmcl=%s&live=yes&_pgSz=400&_lcl=en&_fmt=jsonp&_clbk=W360b23205d13a5cdd46c315ae9c2097b829cbc909" %lg_id

page = requests.get(url, headers={'referer': 'scoresway.com'})
soup = BeautifulSoup(page.content, "html.parser")


raw = soup.getText()
start = raw.find('{"match"')+9
output = raw[start:-2]

#from pandas import json_normalize
#dict = json.loads(output)
#df2 = json_normalize(dict)
#print(df2)
#df2['matchInfo.localDate']
options = webdriver.ChromeOptions()
prefs = {"download.default_directory": lg_schedule_location}
options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(options=options)
driver.get("https://konklone.io/json")
input_css = 'body > section.json > div.areas > textarea'

# Connecting to it with our driver
#input_area = driver.find_element_by_css_selector(input_css)
input_area = driver.find_element(By.CSS_SELECTOR, input_css)#
# Set the sentence into the clipboard
clipboard.copy(output)
# Making sure that there is no previous text
input_area.clear()
# Pasting the copied sentence into the input_area
input_area.send_keys(Keys.SHIFT, Keys.INSERT)

# CSS of the download button
click_css = 'body > section.csv > p > span.rendered > a.download'

# Click it
#driver.find_element_by_css_selector(click_css).click()
driver.find_element(By.CSS_SELECTOR, click_css).click()
time.sleep(2)
driver.close()

path = f'{lg_schedule_location}'
os.chdir(path)
files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
os.rename(files[len(files)-1], '%s Matches.csv' %lg)
print('DONE WITH MATCHES FILE')


################################################################################################################
################################################################################################################
################################################################################################################
################################################################################################################


df = pd.read_csv('%s/%s Matches.csv' %(lg_schedule_location,lg))
df.to_csv('%s/%s Matches.csv' %(lg_schedule_location,lg), encoding='utf-8-sig')


df = pd.read_csv('%s/%s Matches.csv' %(lg_schedule_location,lg))

df.dropna(subset=['liveData/matchDetails/scores/ft/home'],inplace=True)
df.reset_index(drop=True,inplace=True)
df = df[::-1].reset_index(drop=True)

cols = ['matchInfo/localDate','matchInfo/id',
        'matchInfo/contestant/0/officialName','liveData/matchDetails/scores/ft/home',
        'liveData/matchDetails/scores/ft/away','matchInfo/contestant/1/officialName',
        'matchInfo/contestant/0/id','matchInfo/contestant/1/id']
matches = df[cols]

matches['matchName'] = ''
for i in range(len(matches)):
    #matches['matchName'][i] = '%s %i-%i %s' %(matches['matchInfo/contestant/0/officialName'][i], matches['liveData/matchDetails/scores/ft/home'][i], matches['liveData/matchDetails/scores/ft/away'][i], matches['matchInfo/contestant/1/officialName'][i])
    #matches['matchName'][i] = '%s - %s' % (matches['matchInfo/contestant/0/officialName'][i], matches['matchInfo/contestant/1/officialName'][i])
    matches['matchName'][i] = f"{matches['matchInfo/localDate'][i]}_{matches['matchInfo/contestant/0/officialName'][i]} - {matches['matchInfo/contestant/1/officialName'][i]}"
if last_run != '':
    matches['matchInfo/localDate'] = pd.to_datetime(matches['matchInfo/localDate'])
    matches = matches[matches['matchInfo/localDate']>pd.to_datetime(last_run)].reset_index(drop=True)
idd = '57d4g9sd8u1salvv4eip2jhn8'
for i in range(len(matches)):
    url = "https://api.performfeeds.com/soccerdata/matchevent/qxcx5jmswgto1qeqcjzghtddt/%s?_rt=c&_lcl=en&_fmt=jsonp&_clbk=W3bd804271ec7f49caa576ff8367109721760953b4" %matches['matchInfo/id'][i]
    #url = f"https://api.performfeeds.com/soccerdata/matchevent/qxcx5jmswgto1qeqcjzghtddt/{idd}?_rt=c&_lcl=en&_fmt=jsonp&_clbk=W3bd804271ec7f49caa576ff8367109721760953b4"

    page = requests.get(url, headers={'referer': 'scoresway.com'})
    soup = BeautifulSoup(page.content, "html.parser")


    raw = soup.getText()
    start = raw.find("liveData")+10
    output = raw[start:-2]

    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": "%s/%s" %(match_download_location,lg)}
    options.add_experimental_option("prefs",prefs)

    # driver = webdriver.Chrome(chrome_options=options)
    # driver.get("https://konklone.io/json")
    # input_css = 'body > section.json > div.areas > textarea'
    #
    # # Connecting to it with our driver
    # #input_area = driver.find_element_by_css_selector(input_css)
    # input_area = driver.find_element(By.CSS_SELECTOR, input_css)
    # # Set the sentence into the clipboard
    # clipboard.copy(output)
    # # Making sure that there is no previous text
    # input_area.clear()
    # # Pasting the copied sentence into the input_area
    # input_area.send_keys(Keys.SHIFT, Keys.INSERT)
    #
    # # CSS of the download button
    # click_css = 'body > section.csv > p > span.rendered > a.download'
    #
    # # Click it
    # #driver.find_element_by_css_selector(click_css).click()
    # driver.find_element(By.CSS_SELECTOR, click_css).click()
    # time.sleep(2)
    # driver.close()
    #
    path = '%s/%s' %(match_download_location,lg)
    os.chdir(path)
    files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
    #os.rename(files[len(files)-1], '%s.csv' %matches.matchName[i])
    import json

    output1 = json.loads(output)

    with open('data.json', 'w', encoding='utf-8') as f:
        json.dump(output1, f, ensure_ascii=True, indent=4)
    os.rename("data.json", '%s.json' % matches.matchName[i].replace('/',''))
    print('downloaded - %s' %matches.matchName[i])
print('DONE WITH EVERYTHING')

DONE WITH MATCHES FILE


/var/folders/vh/gd9x5_d94g33w411t0md800c0000gn/T/ipykernel_12011/951968058.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matchName'] = ''
/var/folders/vh/gd9x5_d94g33w411t0md800c0000gn/T/ipykernel_12011/951968058.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matchName'][i] = f"{matches['matchInfo/localDate'][i]}_{matches['matchInfo/contestant/0/officialName'][i]} - {matches['matchInfo/contestant/1/officialName'][i]}"
/var/folders/vh/gd9x5_d94g33w411t0md800c0000gn/T/ipykernel_12011/951968058.py:124: SettingWithCopyWarning: 
A valu

downloaded - 2021-12-03_Wellington Phoenix FC - Western Sydney Wanderers FC
downloaded - 2021-12-03_Canberra United - Melbourne City FC
downloaded - 2021-12-04_Sydney FC - Newcastle Jets
downloaded - 2021-12-04_Perth Glory FC - Brisbane Roar FC
downloaded - 2021-12-05_Melbourne Victory - Adelaide United
downloaded - 2021-12-10_Newcastle Jets - Wellington Phoenix FC
downloaded - 2021-12-10_Brisbane Roar FC - Perth Glory FC
downloaded - 2021-12-11_Western Sydney Wanderers FC - Sydney FC
downloaded - 2021-12-12_Melbourne City FC - Melbourne Victory
downloaded - 2021-12-12_Adelaide United - Canberra United
downloaded - 2021-12-17_Western Sydney Wanderers FC - Newcastle Jets
downloaded - 2021-12-18_Melbourne City FC - Adelaide United
downloaded - 2021-12-18_Brisbane Roar FC - Melbourne Victory
downloaded - 2021-12-19_Sydney FC - Wellington Phoenix FC
downloaded - 2021-12-23_Canberra United - Brisbane Roar FC
downloaded - 2021-12-26_Melbourne Victory - Melbourne City FC
downloaded - 2021-12-

## 